In [12]:
import numpy as np
from keras_transformer import get_model, decode

2022-10-14 16:17:19.565232: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-14 16:17:19.565267: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
with open('../Dataset/English_Hindi_Hinglish.txt', mode = 'r') as f:
    data = f.readlines()

data = data[0:55] # 55 Because we have that many labeled data points for Hinglish to English translation.

source_tokens = [i.split(',')[1].strip().split(' ') for i in data]
target_tokens = [i.split('	')[0].strip().split(' ') for i in data]

In [18]:
# Generate dictionaries
def build_token_dict(token_list):
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2,
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict

source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}

# Add special tokens
encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]

# Padding
source_max_len = max(map(len, encode_tokens))
target_max_len = max(map(len, decode_tokens))

encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
decode_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decode_tokens]
decode_output = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

# Build & fit model
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=32,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=False,  # Use different embeddings for different languages
)
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Encoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Encoder-Token-Embedding (Embed  [(None, None, 32),  3168        ['Encoder-Input[0][0]']          
 dingRet)                        (99, 32)]                                                        
                                                                                                  
 Encoder-Embedding (TrigPosEmbe  (None, None, 32)    0           ['Encoder-Token-Embedding[0][0]']
 dding)                                                                                           
                                                                                            

In [19]:
%%time
model.fit(
    x=[np.array(encode_input * 1024), np.array(decode_input * 1024)],
    y=np.array(decode_output * 1024),
    epochs=10,
    batch_size=32,
)

Epoch 1/10
1760/1760 [==============================] - 46s 20ms/step - loss: 0.2883
Epoch 2/10
1760/1760 [==============================] - 35s 20ms/step - loss: 0.0089
Epoch 3/10
1760/1760 [==============================] - 34s 19ms/step - loss: 0.0059
Epoch 4/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0054
Epoch 5/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0075
Epoch 6/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0054
Epoch 7/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0052
Epoch 8/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0052
Epoch 9/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0071
Epoch 10/10
1760/1760 [==============================] - 33s 19ms/step - loss: 0.0056
CPU times: user 11min 31s, sys: 56 s, total: 12min 27s
Wall time: 5min 48s


In [20]:
# Predict
decoded = decode(
    model,
    encode_input,
    start_token=target_token_dict['<START>'],
    end_token=target_token_dict['<END>'],
    pad_token=target_token_dict['<PAD>'],
)
for i in decoded:
    print(' '.join(map(lambda x: target_token_dict_inv[x], i[1:-1])))

Wow!
Help!
Jump.
Jump.
Jump.
Hello!
Hello!
Cheers!
Cheers!
Got it?
I'm OK.
Awesome!
Come in.
Get out!
Go away!
Goodbye!
Perfect!
Perfect!
Welcome.
Welcome.
Have fun.
Have fun.
Have fun.
I forgot.
I forgot.
I'll pay.
I'm fine.
I'm full.
Let's go!
Answer me.
Birds fly.
Excuse me.
Fantastic!
I fainted.
I fear so.
I laughed.
I'm bored.
I'm broke.
I am tired.
It's cold.
Who knows?
Who knows?
Who knows?
Who knows?
Wonderful!
Birds sing.
Come on in.
Definitely!
Don't move.
Fire burns.
Follow him.
I am tired.
I can swim.
I can swim.
I love you.
